In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchaudio.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import string

from datetime import datetime

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# Get the data

## Download dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("Seferovic/bosnian-news-articles-dataset-from-klixba")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/348M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/310M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/786755 [00:00<?, ? examples/s]

In [ ]:
df = ds['train'].to_pandas()

In [ ]:
df.head()

,title,link,article_class,article_class_name,num_of_comments,num_of_shares,picture_path,text
0,Ukrajinski piloti započeli obuku za upravljanj...,https://www.klix.ba/vijesti/svijet/ukrajinski-...,vijesti,Obučavaju ih Amerikanci,0,0,https://static.klix.ba/media/images/vijesti/b_...,Ukrajinski piloti započeli su zajedničku obuku...
1,Košarkaši BiH se danas protiv Poljske bore za ...,https://www.klix.ba/sport/kosarka/kosarkasi-bi...,sport,Finale pretkvalifikacija,8,0,https://static.klix.ba/media/images/vijesti/b_...,Košarkaška reprezentacija Bosne i Hercegovine ...
2,Nakon više od 80 godina Kaliforniji se sprema ...,https://www.klix.ba/vijesti/svijet/nakon-vise-...,vijesti,Uragan Hilary,17,18,https://static.klix.ba/media/images/vijesti/b_...,Uragan Hilary koji se kreće prema pacifičkoj o...
3,Kremlj je na popis stranih agenata u Rusiji uv...,https://www.klix.ba/vijesti/svijet/kremlj-je-n...,vijesti,Paranoja u Moskvi,4,27,https://static.klix.ba/media/images/vijesti/23...,Rusko ministarstvo pravde uključilo je na tako...
4,Savo Manojlović odgovorio Ani Brnabić: Da li s...,https://www.klix.ba/vijesti/regija/savo-manojl...,vijesti,Pitanje odgovornosti,8,8,https://static.klix.ba/media/images/vijesti/b_...,"Direktor pokreta ""Kreni-Promeni"" Savo Manojlov..."


In [ ]:
text = df['text'].values[:1500]

In [ ]:
text = ''.join(text)
text = text[:2_050_000].lower()

In [ ]:
len(text)

2050004

## Prepare dataset

In [ ]:
accepted_text= string.ascii_lowercase + 'šđžčć' + string.digits + '!?.,"()+-/@%–' + "':" + ' ' + '\n'
chars = [x for x in accepted_text]
print(chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'š', 'đ', 'ž', 'č', 'ć', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '?', '.', ',', '"', '(', ')', '+', '-', '/', '@', '%', '–', "'", ':', ' ', '\n']


In [ ]:
text = ''.join([char for char in text if char in accepted_text])

In [ ]:
test_size = 0.20
train_text = text[:int(len(text)*(1-test_size))]
test_text = text[int(len(text)*(1-test_size)):]

In [ ]:
len(train_text), len(test_text)

(1637311, 409328)

## Create vocab

In [ ]:
vocab={}

In [ ]:
for i, char in enumerate(chars):
  vocab[char] = i

In [ ]:
vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 58


## Create Custom tokenizer

In [ ]:
class Tokenizer(object):
  def __init__(self,vocabulary):
    self.vocabulary=vocabulary

  def __call__(self,sequence, label=False):
    sequence = sequence.lower()
    if label:
      token = F.one_hot(torch.tensor(self.vocabulary[sequence]),num_classes=len(self.vocabulary))
      return token.type(torch.float32)
    tokens =[]
    for letter in sequence:
      tokens.append(self.vocabulary[letter])
    return torch.tensor(tokens)

In [ ]:
tokenizer = Tokenizer(vocab)

## Create Custom dataset object

In [ ]:
class TextDataset(Dataset):
  def __init__(self,text,T,tokenizer=None):
    self.tokenizer = tokenizer
    self.text = text
    self.T = T


  def __len__(self):
    return len(self.text)-self.T

  def __getitem__(self,idx):
    x = self.text[idx:idx+self.T]
    y = self.text[idx+self.T]

    if self.tokenizer:
      x = self.tokenizer(x)
      y = self.tokenizer(y, label=True)


    #x = x.astype(np.float32)
   # y = np.array(y).reshape(-1,1).astype(np.float32)
    return x.long(), y

## Load datasets

In [ ]:
train_dataset = TextDataset(train_text,150,tokenizer=tokenizer)
test_dataset = TextDataset(test_text,150,tokenizer=tokenizer)

In [ ]:
len(train_dataset), len(test_dataset)

(1637161, 409178)

In [ ]:
for i,k in train_dataset:
  print(i,k)
  break

tensor([20, 10, 17,  0,  9,  8, 13, 18, 10,  8, 56, 15,  8, 11, 14, 19,  8, 56,
        25,  0, 15, 14, 29,  4, 11,  8, 56, 18, 20, 56, 25,  0,  9,  4,  3, 13,
         8, 29, 10, 20, 56, 14,  1, 20, 10, 20, 56, 25,  0, 56, 20, 15, 17,  0,
        21, 11,  9,  0, 13,  9,  4, 56,  1, 14, 17,  1,  4, 13,  8, 12, 56,  0,
        21,  8, 14, 13,  8, 12,  0, 56,  5, 49, 32, 37, 56, 18,  0, 56,  0, 12,
         4, 17,  8, 29, 10,  8, 12, 56,  8, 13, 18, 19, 17, 20, 10, 19, 14, 17,
         8, 12,  0, 56, 20, 56, 20, 10, 17,  0,  9,  8, 13,  8, 43, 56, 14, 21,
        20, 56,  8, 13,  5, 14, 17, 12,  0,  2,  8,  9, 20, 56,  9,  4, 56, 25,
         0, 56, 12,  4,  3,  8]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


## Initilize DataLoaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for inputs, targets in train_loader:
  print('inputs: ', inputs, 'shape: ', inputs.shape)
  print('targets: ', targets, 'shape: ', targets.shape)
  break

inputs:  tensor([[56, 11,  4,  ..., 12,  0, 56],
        [ 4, 20, 19,  ..., 11,  9,  8],
        [14, 18, 19,  ..., 20,  9, 20],
        ...,
        [56, 45, 18,  ...,  7, 45, 44],
        [32, 34, 44,  ..., 32, 35, 56],
        [21,  8, 11,  ...,  4, 17,  6]]) shape:  torch.Size([32, 150])
targets:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) shape:  torch.Size([32, 58])


# Create the model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
class RNN(nn.Module):
  def __init__(self,vocab_size,embed_size,n_hidden,n_layers,n_outputs):
    super(RNN,self).__init__()
    self.V = vocab_size
    self.D = embed_size
    self.M = n_hidden
    self.L = n_layers
    self.K = n_outputs

    self.embedding = nn.Embedding(self.V,self.D)

    # Initalize rnn and fc layers
    self.rnn = nn.LSTM(input_size=self.D,
                      hidden_size=self.M,
                      num_layers=self.L,
                      batch_first=True)

    self.fc = nn.Sequential(
          nn.Linear(self.M, 1024),
          nn.ReLU(),
          nn.Linear(1024, 512),
          nn.ReLU(),
          nn.Linear(512,self.K)
        )

  def forward(self, X):
    h0 = torch.zeros(self.L,X.size(0),self.M).to(device)
    c0 = torch.zeros(self.L,X.size(0),self.M).to(device)

    # Embedding layer:

    out = self.embedding(X)
    # pass through rnn

    out,_ = self.rnn(out,(h0,c0))
    out = F.relu(out)
    out = self.fc(out[:,-1,:])
    return out


In [ ]:
model = RNN(vocab_size=vocab_size,
            embed_size=40,
            n_hidden=256,
            n_layers=4,
            n_outputs=vocab_size)
model.to(device)

RNN(
  (embedding): Embedding(58, 40)
  (rnn): LSTM(40, 256, num_layers=4, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=58, bias=True)
  )
)

In [ ]:
len(vocab)

58

In [ ]:
for i, k in train_loader:
  print(i.shape)
  tada = model(i.to(device))
  print(tada.size())
  break

torch.Size([32, 150])
torch.Size([32, 58])


# Train model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
def save_model_checkpoint(val_loss, best_val_loss, train_loss, it, model=model, optimizer=optimizer,path='model_checkpoint.pt'):
    torch.save({
            'epoch': it,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            }, path)
    print(f'Model saved at epoch: {it+1}, val_loss improved from: {best_val_loss:.4f} to: {val_loss:.4f}')
best_val_loss=float('inf')

In [ ]:
def plot_loss_curves(train_losses,val_losses,train_losses2=None,val_losses2=None):
  if(train_losses2 is not None):
    train_losses = np.concatenate((train_losses,train_losses2))
    val_losses = np.concatenate((val_losses,val_losses2))
  plt.plot(train_losses,label='Train loss')
  plt.plot(val_losses,label='Val loss')
  plt.legend(); plt.show

In [ ]:
epoches=30
train_losses = np.zeros(epoches)
val_losses = np.zeros(epoches)

for it in range(epoches):
  t0 = datetime.now()
  model.train() # set model to train mode
  train_loss=[]
  val_loss=[]

  # train
  for inputs,targets in train_loader:
    # move data to gpu
    inputs,targets = inputs.to(device),targets.to(device)
    #inputs = inputs.permute(0,2,1)
    # zero gradients
    optimizer.zero_grad()

    # forward pass
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    # backward
    loss.backward()
    optimizer.step()

    train_loss.append(loss.item())

  model.eval() # set model to eval mode
  for inputs,targets in test_loader:
    # move data to gpu
    inputs,targets = inputs.to(device),targets.to(device)
   # inputs = inputs.permute(0,2,1)


    # forward pass
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    val_loss.append(loss.item())

  # calculate loss
  train_loss = np.mean(train_loss)
  val_loss = np.mean(val_loss)
  if (val_loss<best_val_loss):
    save_model_checkpoint(val_loss,best_val_loss,train_loss,it, path='model_checkpoint.pt')
    best_val_loss=val_loss
  # append loss
  train_losses[it]=train_loss
  val_losses[it]=val_loss
  dt = datetime.now() - t0
  print(f"Epoch {it+1}/{epoches}, Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Duration: {dt}")
  print('-------------------------------------------------------------')

Model saved at epoch: 1, val_loss improved from: inf to: 1.5736
Epoch 1/30, Train loss: 1.7885, Val loss: 1.5736, Duration: 0:18:33.757242
-------------------------------------------------------------
Model saved at epoch: 2, val_loss improved from: 1.5736 to: 1.4906
Epoch 2/30, Train loss: 1.4877, Val loss: 1.4906, Duration: 0:18:32.034035
-------------------------------------------------------------
Model saved at epoch: 3, val_loss improved from: 1.4906 to: 1.4563
Epoch 3/30, Train loss: 1.4152, Val loss: 1.4563, Duration: 0:18:27.215023
-------------------------------------------------------------
Model saved at epoch: 4, val_loss improved from: 1.4563 to: 1.4315
Epoch 4/30, Train loss: 1.3720, Val loss: 1.4315, Duration: 0:18:29.404735
-------------------------------------------------------------
Model saved at epoch: 5, val_loss improved from: 1.4315 to: 1.4184
Epoch 5/30, Train loss: 1.3412, Val loss: 1.4184, Duration: 0:18:34.244518
---------------------------------------------

In [ ]:
print_second = False
if print_second:
  plot_loss_curves(train_losses2,val_losses2,train_losses,val_losses,)
else:
  plot_loss_curves(train_losses,val_losses)
  train_losses2,val_losses2 = train_losses,val_losses

In [ ]:
model.eval()
# train accuracy
n_correct=0
n_total=0
for inputs,targets in train_loader:
  # move data to gpu
  inputs,targets = inputs.to(device),targets.to(device)
  #inputs = inputs.permute(0,2,1)

  # make prediction
  outputs = model(inputs)
  _,predictions = torch.max(outputs,1)
  targets = torch.argmax(targets, dim=1)
  # update counts
  n_correct+=(predictions==targets).sum().item()
  n_total+=targets.shape[0]
train_acc = n_correct/n_total

# test accuracy
n_correct=0
n_total=0
for inputs,targets in test_loader:
  # move data to gpu
  inputs,targets = inputs.to(device),targets.to(device)
  #inputs = inputs.permute(0,2,1)

  # make prediction
  outputs = model(inputs)
  _,predictions = torch.max(outputs,1)
  targets = torch.argmax(targets, dim=1)
  # update counts
  n_correct+=(predictions==targets).sum().item()
  n_total+=targets.shape[0]
test_acc = n_correct/n_total

print(f"Train accuracy: {train_acc:.4f}, Test accuracy: {test_acc:.4f}")

Train accuracy: 0.6215, Test accuracy: 0.5521


In [ ]:
vocabulary = {y: x for x, y in vocab.items()}

In [ ]:
import time

In [ ]:
tekst = text[500000:500100]
tekst='FK Željezničar je uoči sjednice obavijestio medije da'
#tekst = '''Apsolutni junak Zmajeva večeras je bio golman Nikola Vasilj, koji je sa pet izvanrednih intervencija sačuvao svoju mrežu netaknutom, pa je najzaslužniji za osvojeni bod našeg nacionalnog tima.#
#Od početka utakmice inicijativu je imala selekcija Mađarske, ali ipak se dugo čekalo na prava uzbuđenja jer smo gledali uspavanku na terenu.'''
for i in range(50):
  data = tokenizer(tekst[-100:])
  data = data.reshape(1,-1)
  data = data.to(device)
  outputs = model(data)
  out = torch.argmax(outputs,1)
  new_letter = vocabulary[out.cpu().numpy()[0]]
  tekst = tekst+new_letter
  print(tekst)
  time.sleep(0.5)
  print('---------------------------------------------------')
#print(tekst)

FK Željezničar je uoči sjednice obavijestio medije da 
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da s
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se 
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se n
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se na
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se nad
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se nada
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije da se nada 
---------------------------------------------------
F

In [ ]:
def next_char(text, temperature=1):
    # Predict using the model (assuming text is already processed into the right tensor format)
    with torch.no_grad():
      data = tokenizer(text)
      data = data.reshape(1,-1)
      data = data.to(device)
      logits = model(data)  # Replace with proper text input processing

    # Get the logits for the last predicted character
    #logits = y_proba[:, -1, :]  # Assuming y_proba has shape [batch_size, seq_len, vocab_size]

    # Rescale logits using temperature
    rescaled_logits = logits / temperature

    # Apply softmax to get probabilities and then sample from the categorical distribution
    probabilities = F.softmax(rescaled_logits, dim=-1)
    char_id = torch.multinomial(probabilities, num_samples=1).item()

    # Get the vocabulary and return the corresponding character
    return vocabulary[char_id]

In [ ]:
def extend_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tekst = text[500000:500100]

print(extend_text(tekst, temperature=1))

z vjenčanog prstena te je na taj način potvrdila da je vijest o razvodu tačna. ona je u aprilu bila u zajednici. tarocinov materijalna fodica (društvo i gonky ne svaki 10 mjesta ispred svog nevjetove 


In [ ]:
tekat="""Ali, želja Ukrajine za članstvom u EU posebno je stvorila opipljiv strah na zapadnom Balkanu da će biti ostavljena po strani. Srbija ne želi da ima ništa sa NATO-om, a njen blizak odnos sa Moskvom zakomplikovao je nastojanja Beograda za ulazak u EU, još više od ruske invazije velikih razmjera na Ukrajinu.

Da nije bilo invazije na Ukrajinu, pristupni pregovori sa Albanijom i Sjevernom Makedonijom zasigurno bi zaglavili, a Bosna i Hercegovina ne bi bila priznata kao kandidat za EU. Možda se i EU ne bi složila oko budžeta za svoj novi plan rasta od šest milijardi eura za Zapadni Balkan. Plan uslovljava evropska ulaganja reformama na Balkanu, ali ako se ostvari njegov puni potencijal, zemlje u regionu mogle bi dobiti po glavi stanovnika skoro onoliko nov"""
print(extend_text(tekat, temperature=0.5))

Ali, želja Ukrajine za članstvom u EU posebno je stvorila opipljiv strah na zapadnom Balkanu da će biti ostavljena po strani. Srbija ne želi da ima ništa sa NATO-om, a njen blizak odnos sa Moskvom zakomplikovao je nastojanja Beograda za ulazak u EU, još više od ruske invazije velikih razmjera na Ukrajinu.

Da nije bilo invazije na Ukrajinu, pristupni pregovori sa Albanijom i Sjevernom Makedonijom zasigurno bi zaglavili, a Bosna i Hercegovina ne bi bila priznata kao kandidat za EU. Možda se i EU ne bi složila oko budžeta za svoj novi plan rasta od šest milijardi eura za Zapadni Balkan. Plan uslovljava evropska ulaganja reformama na Balkanu, ali ako se ostvari njegov puni potencijal, zemlje u regionu mogle bi dobiti po glavi stanovnika skoro onoliko novi dan provjere je i dobio je potpis u karakteru, a nakon što je na veliki prijenos i naših organizac
